In [1]:
import os
import shutil
import pydub
import matplotlib.pyplot as plt

In [3]:
def get_prober_name():
    return "C://ffmpeg/bin/ffprobe.exe"


pydub.AudioSegment.converter = "C://ffmpeg/bin/ffmpeg.exe"                  
pydub.utils.get_prober_name = get_prober_name

In [2]:
shutil.copytree('io/raw_audios', 'io/audios', dirs_exist_ok=True)

'audios'

In [ ]:
audio_1 = pydub.AudioSegment.from_file("io/audios/Khadijeh_1.ogg")
audio_2 = pydub.AudioSegment.from_file("io/audios/Khadijeh_2.ogg")

result = audio_1 + audio_2
result.export("io/audios/Khadijeh.ogg")

os.remove("io/audios/Khadijeh_1.ogg")
os.remove("io/audios/Khadijeh_2.ogg")

In [21]:
audio_1 = pydub.AudioSegment.from_file("io/audios/Parsa_1.ogg")
audio_2 = pydub.AudioSegment.from_file("io/audios/Parsa_2.ogg")

result = audio_1 + audio_2
result.export("io/audios/Parsa.ogg")

os.remove("io/audios/Parsa_1.ogg")
os.remove("io/audios/Parsa_2.ogg")

In [24]:
audio_1 = pydub.AudioSegment.from_file("io/audios/Nima_1.ogg")
audio_2 = pydub.AudioSegment.from_file("io/audios/Nima_2.ogg")

result = audio_1 + audio_2
result.export("io/audios/Nima.ogg")

os.remove("io/audios/Nima_1.ogg")
os.remove("io/audios/Nima_2.ogg")

In [26]:
audio = pydub.AudioSegment.from_file("io/Mona.m4a")

audio.export("io/Mona.ogg", format='ogg')
os.remove("io/Mona.m4a")

In [33]:
show_plot = False

for file_name in os.listdir("io/audios"):
    file_path = os.path.join("io/audios", file_name)
    audio = pydub.AudioSegment.from_file(file_path)

    if show_plot:
        audio_array = audio.get_array_of_samples()
        plt.figure(figsize=(20, 2))
        plt.plot(audio_array)
        plt.title(file_name)
        plt.show()

    # preprocessing
    audio = audio.set_sample_width(2)  # convert to 16-bit (2 bytes per sample) to avoid tensorflow error
    audio = audio.set_frame_rate(48000)
    audio = audio.set_channels(1)  # convert stereo audio to mono audio to avoid tensorflow error

    chunks = pydub.silence.split_on_silence(audio, min_silence_len=900, silence_thresh=-45)
    result = sum(chunks)

    person_name = file_name.split(".")[0]
    result_path = os.path.join("io/audios", person_name+".wav")
    
    # save wav file and remove ogg file
    result.export(result_path, format="wav")
    os.remove(file_path)

In [34]:
for file_name in os.listdir("io/audios"):
    file_path = os.path.join("io/audios", file_name)
    audio = pydub.AudioSegment.from_file(file_path)

    person_name = file_name.split(".")[0]
    os.makedirs(os.path.join("io/dataset", person_name), exist_ok=True)

    chunks = pydub.utils.make_chunks(audio, 1000)
    for i, chunk in enumerate(chunks):
        if len(chunk) >= 1000:
            result_path = os.path.join("io/dataset", person_name, f"voice_{i}.wav")
            chunk.export(result_path, format="wav")